# Inlämning 2: Numpy, Pandas och Analys

### Part 1: Numpy

In [94]:
import numpy as np
np.set_printoptions(precision=4, suppress=True)

Konstruera en array med 10 kolumner och 5 rader av float-tal. Inget tal får vara samma.

In [95]:
first_array = np.random.normal(size=(5, 10))

print(first_array)
print(first_array.dtype)

[[-0.0665 -0.5496 -1.1417  0.4805  0.6961 -0.986   1.2317  0.7997 -0.6458
  -0.0767]
 [-0.3007  1.4283  0.2297 -0.7057 -0.0135 -0.7161  1.1202 -0.6132  0.8562
   0.8858]
 [-0.6429  0.3634 -0.3429  0.8895 -2.7259 -0.4941  1.3848 -0.2747  1.3156
   0.1955]
 [-0.7405  1.8126 -0.1455  0.3053  0.6859 -1.1252  1.0617 -0.1045  0.5534
   0.1175]
 [-1.2781 -1.1834 -1.9604  0.1004  0.2012 -0.815   0.8062  1.0162 -0.5733
   1.2464]]
float64


Konstruera en ny array till med 10 kolumner och 7 rader av float-tal. Den skall inte vara samma som ovan array..

In [96]:
second_array = np.random.randn(7, 10)

print(second_array)
print(second_array.dtype)

[[ 1.2495 -1.286   1.5343  1.9211 -0.0483  1.3025  1.4859  0.0435 -0.6179
   1.1917]
 [ 2.1232 -0.3088 -0.9616 -0.8796  0.8695 -0.379  -1.0362  0.179   0.0431
  -2.5949]
 [ 1.2366 -0.5971 -0.1366 -2.2703 -0.2796  0.2856 -0.4027 -0.5865 -0.3353
   1.3346]
 [-0.4378 -0.7518  1.4264 -1.8497 -0.3829  0.9832 -0.9724  1.052  -1.3239
   0.6371]
 [-1.5124 -0.8172  0.1707 -0.341   0.0546 -0.5442 -0.057   1.4326  1.258
   1.1272]
 [-0.3006  1.853   0.3346  1.1716  1.0663 -0.5646 -1.2079 -1.045   1.2922
  -0.1179]
 [ 1.0881 -1.0125 -0.7663 -0.4981 -0.7864  0.9923  0.8291  1.9148  0.3624
  -1.1774]]
float64


Gör om de ovan arrayerna så att de har 5 kolumner men innehåller samma data.

In [97]:
print(first_array)
print(second_array)
first_array = np.delete(first_array, slice(5, 10), axis=1)
second_array = np.delete(second_array, slice(5, 10), axis=1)
print(first_array)
print(second_array)

[[-0.0665 -0.5496 -1.1417  0.4805  0.6961 -0.986   1.2317  0.7997 -0.6458
  -0.0767]
 [-0.3007  1.4283  0.2297 -0.7057 -0.0135 -0.7161  1.1202 -0.6132  0.8562
   0.8858]
 [-0.6429  0.3634 -0.3429  0.8895 -2.7259 -0.4941  1.3848 -0.2747  1.3156
   0.1955]
 [-0.7405  1.8126 -0.1455  0.3053  0.6859 -1.1252  1.0617 -0.1045  0.5534
   0.1175]
 [-1.2781 -1.1834 -1.9604  0.1004  0.2012 -0.815   0.8062  1.0162 -0.5733
   1.2464]]
[[ 1.2495 -1.286   1.5343  1.9211 -0.0483  1.3025  1.4859  0.0435 -0.6179
   1.1917]
 [ 2.1232 -0.3088 -0.9616 -0.8796  0.8695 -0.379  -1.0362  0.179   0.0431
  -2.5949]
 [ 1.2366 -0.5971 -0.1366 -2.2703 -0.2796  0.2856 -0.4027 -0.5865 -0.3353
   1.3346]
 [-0.4378 -0.7518  1.4264 -1.8497 -0.3829  0.9832 -0.9724  1.052  -1.3239
   0.6371]
 [-1.5124 -0.8172  0.1707 -0.341   0.0546 -0.5442 -0.057   1.4326  1.258
   1.1272]
 [-0.3006  1.853   0.3346  1.1716  1.0663 -0.5646 -1.2079 -1.045   1.2922
  -0.1179]
 [ 1.0881 -1.0125 -0.7663 -0.4981 -0.7864  0.9923  0.8291  1.9148

Slå ihop de bägge arrayerna till en array.

In [98]:
result_array = np.concatenate((first_array, second_array), axis=0)
print(result_array)

[[-0.0665 -0.5496 -1.1417  0.4805  0.6961]
 [-0.3007  1.4283  0.2297 -0.7057 -0.0135]
 [-0.6429  0.3634 -0.3429  0.8895 -2.7259]
 [-0.7405  1.8126 -0.1455  0.3053  0.6859]
 [-1.2781 -1.1834 -1.9604  0.1004  0.2012]
 [ 1.2495 -1.286   1.5343  1.9211 -0.0483]
 [ 2.1232 -0.3088 -0.9616 -0.8796  0.8695]
 [ 1.2366 -0.5971 -0.1366 -2.2703 -0.2796]
 [-0.4378 -0.7518  1.4264 -1.8497 -0.3829]
 [-1.5124 -0.8172  0.1707 -0.341   0.0546]
 [-0.3006  1.853   0.3346  1.1716  1.0663]
 [ 1.0881 -1.0125 -0.7663 -0.4981 -0.7864]]


Ändra ordningen på kolumnerna så att de har ordningen kolumn5, kolumn3, kolumn4, kolumn1 och kolumn2.

In [99]:
out = result_array[:, [4, 2, 3, 0, 1]]
print(out)

[[ 0.6961 -1.1417  0.4805 -0.0665 -0.5496]
 [-0.0135  0.2297 -0.7057 -0.3007  1.4283]
 [-2.7259 -0.3429  0.8895 -0.6429  0.3634]
 [ 0.6859 -0.1455  0.3053 -0.7405  1.8126]
 [ 0.2012 -1.9604  0.1004 -1.2781 -1.1834]
 [-0.0483  1.5343  1.9211  1.2495 -1.286 ]
 [ 0.8695 -0.9616 -0.8796  2.1232 -0.3088]
 [-0.2796 -0.1366 -2.2703  1.2366 -0.5971]
 [-0.3829  1.4264 -1.8497 -0.4378 -0.7518]
 [ 0.0546  0.1707 -0.341  -1.5124 -0.8172]
 [ 1.0663  0.3346  1.1716 -0.3006  1.853 ]
 [-0.7864 -0.7663 -0.4981  1.0881 -1.0125]]


### Part 2: Pandas

In [100]:
import pandas as pd

In [101]:
assults = pd.read_csv('assults.csv')
# print(assults.describe())

##### 1 - Hur många regioner finns med i data?

In [102]:
# print(assults.columns)
unique_region_labels = assults['Region_2013_label'].unique()
# print(unique_region_labels)
print(f'The total number of regions is: {len(unique_region_labels)}')

The total number of regions is: 17


##### 2 - Lista regionerna i bokstavsordning

In [103]:
region_names = unique_region_labels.tolist()
print(sorted(region_names))

['Area Outside Region', 'Auckland Region', 'Bay of Plenty Region', 'Canterbury Region', 'Gisborne Region', "Hawke's Bay Region", 'Manawatu-Wanganui Region', 'Marlborough Region', 'Nelson Region', 'Northland Region', 'Otago Region', 'Southland Region', 'Taranaki Region', 'Tasman Region', 'Waikato Region', 'Wellington Region', 'West Coast Region']


##### 3 - Vad är invånarantalet i varje region

In [104]:
#assults.head(20)
print('Population in mid point 2015:\n')
series_grouped = assults.groupby('Region_2013_label')['Population_mid_point_2015'].sum()
series_grouped

Population in mid point 2015:



Region_2013_label
Area Outside Region             610
Auckland Region             1569890
Bay of Plenty Region         287130
Canterbury Region            586310
Gisborne Region               47370
Hawke's Bay Region           160010
Manawatu-Wanganui Region     234460
Marlborough Region            45340
Nelson Region                 49940
Northland Region             168320
Otago Region                 215030
Southland Region              97190
Taranaki Region              115760
Tasman Region                 49500
Waikato Region               439000
Wellington Region            496890
West Coast Region             32700
Name: Population_mid_point_2015, dtype: int64

##### 4 - Hur många brott begicks i varje region

In [105]:
# assults.head(20)
print('Total number of victims in 2015:\n')
series_grouped = assults.groupby('Region_2013_label')['Victimisations_calendar_year_2015'].sum()
series_grouped

Total number of victims in 2015:



Region_2013_label
Area Outside Region            0
Auckland Region             8010
Bay of Plenty Region        1751
Canterbury Region           2942
Gisborne Region              436
Hawke's Bay Region          1068
Manawatu-Wanganui Region    1270
Marlborough Region           209
Nelson Region                357
Northland Region             905
Otago Region                1142
Southland Region             381
Taranaki Region              493
Tasman Region                161
Waikato Region              2330
Wellington Region           2459
West Coast Region            121
Name: Victimisations_calendar_year_2015, dtype: int64

##### 5 - Skapa en dataframe på de områden med de 10 högsta antalet överfall.

In [106]:
# assults.head()
series_grouped = assults.groupby('Urban_area_2013_label')['Victimisations_calendar_year_2015'].sum()
df_grouped = series_grouped.to_frame(name='Number_of_attacks').reset_index()
df_grouped = df_grouped.sort_values(['Number_of_attacks'], ascending=False).groupby('Urban_area_2013_label').head(10).reset_index()
df_grouped = df_grouped.head(10)
df_grouped


,index,Urban_area_2013_label,Number_of_attacks
0,11,Central Auckland Zone,3286
1,102,Southern Auckland Zone,2770
2,12,Christchurch,2348
3,29,Hamilton Zone,1196
4,98,Rural (Incl.some Off Shore Islands),1056
5,139,Wellington Zone,1019
6,141,Western Auckland Zone,900
7,110,Tauranga,693
8,18,Dunedin,657
9,69,Northern Auckland Zone,635


##### 6 - Lägg till en kolumn i dataframen som visar % antal överfall i området jämfört med invånarantal i området

In [107]:
#assults.head()
def condition(x):
    if x['Victimisations_calendar_year_2015'] != 0 and x['Population_mid_point_2015'] !=0:
        return x['Victimisations_calendar_year_2015'] / x['Population_mid_point_2015'] * 100   
    else:
        return 0

assults_copy = assults
assults_copy['Procent_attacks_per_population'] = assults_copy.apply(condition, axis=1)
assults_copy.head()

,Index,Area_unit_2013_code,Area_unit_2013_label,Victimisations_calendar_year_2015,Population_mid_point_2015,Rate_per_10000_population,Rate_ratio_NZ_average_rate,Urban_area_2013_code,Urban_area_2013_label,Urban_area_type,Territorial_authority_area_2013_code,Territorial_authority_area_2013_label,Region_2013_code,Region_2013_label,Procent_attacks_per_population
0,1,514102,Auckland Central West,550,15900,346,6.6,4,Central Auckland Zone,Main urban area,76,Auckland,2,Auckland Region,3.459119
1,2,573101,Willis Street-Cambridge Terrace,447,8300,539,10.3,20,Wellington Zone,Main urban area,47,Wellington City,9,Wellington Region,5.385542
2,3,529700,Hamilton Central,380,3170,"1,199",22.9,6,Hamilton Zone,Main urban area,16,Hamilton City,3,Waikato Region,11.987382
3,4,514103,Auckland Central East,375,13250,283,5.4,4,Central Auckland Zone,Main urban area,76,Auckland,2,Auckland Region,2.830189
4,5,591500,Cathedral Square,305,570,"5,351",102.3,22,Christchurch,Main urban area,60,Christchurch City,13,Canterbury Region,53.508772


##### 7 - Skapa en ny dataframe som innehåller antalet områden som finns av de olika typerna av urban area. Ledtråd: groupby

In [108]:
# assults.head()
series_grouped = assults.groupby('Urban_area_type').size()
df_grouped = series_grouped.to_frame(name='Number_of_areas').reset_index()
print(df_grouped)

        Urban_area_type  Number_of_areas
0       Main urban area             1112
1      Minor urban area              137
2            Rural area              640
3  Secondary urban area              131


##### 8 - Ta reda på hur många områden hade population men inga överfall. Jämför totala populationen i dessa med totala populationen i deras region.

In [109]:
assults_copy = assults
assults_copy = assults_copy[(assults_copy['Population_mid_point_2015'] > 0)]
assults_copy = assults_copy[(assults_copy['Victimisations_calendar_year_2015'] == 0)].reset_index()
len(assults_copy)

291

In [124]:
assults_copy = assults
assults_copy = assults_copy[(assults_copy['Population_mid_point_2015'] > 0)]
assults_copy = assults_copy[(assults_copy['Victimisations_calendar_year_2015'] == 0)].reset_index()
assults_copy['total_by_region'] = assults_copy['Population_mid_point_2015'].groupby(assults_copy['Region_2013_label']).transform('sum')
df_population_comparison = assults_copy[['Urban_area_2013_label','Population_mid_point_2015', 'Region_2013_label', 'total_by_region']]
df_population_comparison

,Urban_area_2013_label,Population_mid_point_2015,Region_2013_label,total_by_region
0,Inlet-in TA but not in Urban Area,10,Southland Region,14060
1,Inlet-in TA but not in Urban Area,10,Tasman Region,1020
2,Inlet-in TA but not in Urban Area,10,Otago Region,17290
3,Inlet-in TA but not in Urban Area,10,Waikato Region,18780
4,Rural (Incl.some Off Shore Islands),10,Southland Region,14060
...,...,...,...,...
286,Rural (Incl.some Off Shore Islands),3510,Auckland Region,32520
287,Invercargill,3590,Southland Region,14060
288,Tauranga,3590,Bay of Plenty Region,6380
289,Wellington Zone,4020,Wellington Region,36110


##### 9 - Räkna antalet områden som inte har någon data på överfall eller invånare.

In [111]:
assults_copy = assults
assults_copy = assults_copy[(assults_copy['Population_mid_point_2015'] == 0) | (['Victimisations_calendar_year_2015'] == 0)]
print(f'Antalet områden som inte har någon data på överfall eller invånare: {len(assults_copy)}')


Antalet områden som inte har någon data på överfall eller invånare: 126


##### 10 - VG: Skapa en ny dataframe med 10 områderna med mer än 5000 invånare, men har de lägsta överfallen per invånare.

##### 11 - VG: Skapa en ny dataframe som innehåller % överfall i regionen jämfört med antalet invånare i regionen.

##### 12 - VG: Lägg till en ny kolumn i dataframen som visar % antal överfall per region

### Part 3: Utforskning/analys

Minst 500 rader, 5 Kolumner i ditt valda dataset

#### Step 1: Välj dataset

Hitta ett intressant dataset. Det valda datasettet måste vara i CSV format,  ha minst 500 rader och 5 kolumner. 
Ladda ner datasetet med pandas read_csv funktion och en url (se nedan). Alternativt i inlämningen skicka med datasettet.

In [112]:
url = 'https://raw.githubusercontent.com/cs109/2014_data/master/countries.csv'
data = pd.read_csv(url)

#### Step 2: Data preparation och Cleaning

Ladda in dataset in i en dataframe med pandas. 
Behandla missade data, icke-korrekt data och fel data.
Gör andra steg som behövs (gör om sträng datum till riktiga datum, skapa fler kolumner, slå ihop datasets osv).
Summera hur datasetet ser ut i nuvarande läge i dina markdowns. Storlek, kolumner, kategori typer (qualitative vs. quantitative), kvalitet på data, fång osv.

#### Step 3: Undersökande analys och visualisering

Undersök data genom analysera (mean, sum, range osv)  intressant statistik i numeriska kolumnerna.
Undersök distributions av numeriska kolumner genom histogram osv
Undersök relationen mellan kolumner med scatter plots, bar chars osv.
Notera intressanta upptäckter i din markdown.

#### Step 4: För VG - fråga och svara på 4 intressanta frågor om datan

#### Step 5: Summera dina slutsatser & Skriv ett avslut.